In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# function which returns the number of occurnces of each element of an array
def count_occurrences(array):
    
    unique_elements, counts = np.unique(array, return_counts=True)
    
    occurrences = dict(zip(unique_elements, counts))
    
    return occurrences

def max(array):
    
    max_value = array[0]
    
    for i in range(1, len(array)):
        
        if array[i] > max_value:
            
            max_value = array[i]
    
    return max_value

def min(array):
    
    min_value = array[0]
    
    for i in range(1, len(array)):
        
        if array[i] < min_value:
            
            min_value = array[i]
    
    return min_value

# function which returns the local maximums positions and their time value
def local_max(time, position):
    
    max_pos = {}
    
    delta = int(len(position) / 200)
    i = 0
    j = 1
    while i < len(position):
        
        i_min = i - delta
        i_max = i + delta
        
        if (i_min < 0):    
            i_min = 0
        if (i_max > len(position)):
            i_max = len(position)
        
        if (position[i] == max(position[i_min:i_max])):
            
            max_pos[j] = [time[i], position[i]]
            j += 1
            
            i = i_max - 1
        
        i += 1
    
    return max_pos

# funtion which returns the matrix of all the possible periods of oscillations
def periods(maxima):
    
    p_osc = {}
    
    for key_1 in maxima:
        
        for key_2 in maxima:
        
            if (key_2 > key_1):
                
                p_osc[(key_1, key_2)] = ((maxima[key_2][0] - maxima[key_1][0]) / (key_2 - key_1))
    
    return p_osc

# functions which calculates the mean of the elements of a dictionary
def dic_mean(dic):
    
    n = 0
    sum = 0
    
    for key in dic:
        
        sum += dic[key]
        n += 1
    
    return (sum / n)



# analysis function

def analysis(path_dati, path_regressione, type, tool, int_configurazione):
    
    # import csv files for data as DataFrames
    dati = pd.read_csv(path_dati, sep=';')
    # import csv files for regression parameters as DataFrames
    regressione = pd.read_csv(path_regressione, sep=';')
    
    if (type == 'statico'):
        quantity = 'heigth'
        delta = 20
        str_delta = 'Δ'
        xlabel = 'Δh [mm]'
    elif (type == 'dinamico'):
        quantity = 'omega'
        delta = 0.005
        str_delta=''
        xlabel = '1/ω² [s²]'
    else:
        return "Invalid type inserted"
    
    # convert DatFrames of data to numpy arrays
    measure = dati[quantity].to_numpy()
    measure_error = dati['sigma_'+quantity].to_numpy()
    mass = dati['mass'].to_numpy()
    mass_error = dati['sigma_mass'].to_numpy()
    
    # define domain of linear regression
    x = np.linspace(0, max(measure)+delta, 1000)
    
    # plot data and line
    plt.errorbar(measure, mass, mass_error, measure_error, fmt='.', ecolor='red')
    plt.plot(x, (regressione['A'][0] * x + regressione['B'][0]))
    plt.grid()
    plt.xlabel(xlabel)
    plt.ylabel(str_delta+'m [g]')
    plt.title('Linear regression for configuration: '+type+" "+tool+" "+str(int_configurazione))
    plt.savefig("analisi/molla/plot_"+type+"_"+tool+"_"+str(int_configurazione)+".png", dpi=1200)
    plt.close()

analysis('dati/molla/statico_calibro_1.csv', 'dati/molla/regressione_statico_calibro_1.csv', 'statico', 'calibro', 1)
analysis('dati/molla/statico_sensore_1.csv', 'dati/molla/regressione_statico_sensore_1.csv', 'statico', 'sensore', 1)
analysis('dati/molla/statico_calibro_2.csv', 'dati/molla/regressione_statico_calibro_2.csv', 'statico', 'calibro', 2)
analysis('dati/molla/statico_sensore_2.csv', 'dati/molla/regressione_statico_sensore_2.csv', 'statico', 'sensore', 2)
analysis('dati/molla/dinamico_sensore_1.csv', 'dati/molla/regressione_dinamico_sensore_1.csv', 'dinamico', 'sensore', 1)
analysis('dati/molla/dinamico_sensore_2.csv', 'dati/molla/regressione_dinamico_sensore_2.csv', 'dinamico', 'sensore', 2)